In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
y = data['label']

In [28]:
tmp = []
for a,b in zip(data['title'],data['text']):
    tmp.append(a+' '+b)

In [29]:
x = tmp

In [7]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)

In [8]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

def cleaning(sample):
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in sample.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text.lower()
    # Remove links and emails
    text = re.sub(r'http\S+|www\S+|[\w\.-]+@[\w\.-]+', '', text)
    # Remove any thing except words, numbers, and space
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'[0-9]', '', text)
    # Remove emojis and non ascii characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Remove Stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Extract lemmatized tokens
    lem = WordNetLemmatizer()
    lemmatized_tokens = [lem.lemmatize(word) for word in text.split()]
    # Join the lemmatized tokens into a sentence
    text = ' '.join(lemmatized_tokens)
    text = ' '.join([word for word in text.split() if (len(word)>1 or word == 'i')])
    return text.split()

In [31]:
for i in range(len(x)):
    x[i] = cleaning(x[i])
print(x[0])

['smell', 'hillary', 'fear', 'daniel', 'greenfield', 'shillman', 'journalism', 'fellow', 'freedom', 'center', 'new', 'york', 'writer', 'focusing', 'radical', 'islam', 'final', 'stretch', 'election', 'hillary', 'rodham', 'clinton', 'gone', 'war', 'fbi', 'word', 'unprecedented', 'thrown', 'around', 'often', 'election', 'ought', 'retired', 'still', 'unprecedented', 'nominee', 'major', 'political', 'party', 'go', 'war', 'fbi', 'thats', 'exactly', 'hillary', 'people', 'done', 'coma', 'patient', 'waking', 'watching', 'hour', 'cnn', 'hospital', 'bed', 'would', 'assume', 'fbi', 'director', 'james', 'comey', 'hillary', 'opponent', 'election', 'fbi', 'attack', 'everyone', 'obama', 'cnn', 'hillary', 'people', 'circulated', 'letter', 'attacking', 'comey', 'currently', 'medium', 'hit', 'piece', 'lambasting', 'targeting', 'trump', 'wouldnt', 'surprising', 'clinton', 'ally', 'start', 'running', 'attack', 'ad', 'fbi', 'fbi', 'leadership', 'warned', 'entire', 'leftwing', 'establishment', 'form', 'lynch

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)

In [33]:
max_len = 0
for i in x:
    if max_len < len(i):
        max_len = len(i)
x = pad_sequences(x, maxlen=max_len)

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import tensorflow as tf
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
optimzer = tf.keras.optimizers.Adadelta(learning_rate=0.5)
model.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [38]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 104s 633ms/step - accuracy: 0.5255 - loss: 0.6920 - val_accuracy: 0.5280 - val_loss: 0.6694
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 142s 634ms/step - accuracy: 0.7005 - loss: 0.5887 - val_accuracy: 0.8295 - val_loss: 0.3968
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 99s 625ms/step - accuracy: 0.8613 - loss: 0.3509 - val_accuracy: 0.7009 - val_loss: 0.6677
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 142s 628ms/step - accuracy: 0.9022 - loss: 0.2515 - val_accuracy: 0.8635 - val_loss: 0.3251
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 142s 624ms/step - accuracy: 0.9455 - loss: 0.1642 - val_accuracy: 0.8816 - val_loss: 0.3138
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 100s 631ms/step - accuracy: 0.9670 - loss: 0.1000 - val_accuracy: 0.8769 - val_loss: 0.3443
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 142s 630ms/step - accuracy: 0.9832 - loss: 0.0566 - val_accuracy: 0.8682 - val_loss: 0.3245
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 141s 627ms/step - accuracy: 0.9883 - 

In [39]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 222ms/step


In [45]:
y_pred = y_pred.reshape(1,-1)

In [46]:
print(f'Accuracy : {np.sum(y_pred == y_test)/len(y_test)*100}%')

Accuracy : 87.60852407261247%
